# Evaluer la performance du modèle

## Tester la qualité des prédictions

+ Séparer le dataframe en 2: 
 - train_df contenant 6000 lignes de paris_listings
 - test_df contenant le reste
+ Modifier la fonction predict_price, changer le DataFrame temp_df. Changer le DataFrame paris_listings à train_df, pour que seul le dataset de training soit utilisé.
+ Utiliser la méthode Series apply pour appliquer la fonction predict_price  sur les valeurs de la colonne 'accommodates' du DataFrame test_df.
+ Assigner l'objet Series résultant à la colonne 'predicted_price' de test_df.


In [2]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
np.random.seed(1)

paris_listings = pd.read_csv("paris_airbnb.csv")
paris_listings = paris_listings.loc[np.random.permutation(len(paris_listings))]
stripped_commas = paris_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
paris_listings['price'] = stripped_dollars.astype('float')


train_df = paris_listings.iloc[0:6000]
test_df = paris_listings.iloc[6000:]

def predict_price(new_listing):
    temp_df = paris_listings.copy()
    temp_df["distance"] = temp_df["accommodates"].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values("distance")
    return temp_df['price'].iloc[0:5].mean()

test_df['predicted_price'] = test_df['accommodates'].apply(lambda x: predict_price(x))

/var/folders/w7/6rt7vbvd7bqbx0q2zdmnfvm40000gn/T/ipykernel_27364/3800831289.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stripped_dollars = stripped_commas.str.replace('$', '')


## Les métriques d'erreur

+ Utiliser la méthode numpy.absolute() pour calculer l'erreur absolue moyenne MAE entre predicted_price et price.
+ Assigner le résultat à la variable mae.


In [5]:
import numpy as np

test_df['error'] = np.absolute(test_df['predicted_price'] - test_df['price'])
mae = test_df['error'].mean()
print(mae)

49.090099999999744


## Erreur quadratique moyenne

+ Calculer la valeur de MSE entre les colonnes 'predicted_price' et 'price'.
+ Assigner le résultat à la variable mse.
+ Afficher le résultat.


In [6]:
mse = ((test_df['predicted_price'] - test_df['price'])**2).mean()
mse

6762.121780000079

## Entrainer un autre modèle

+ Modifier la fonction suivante predict_price en utilisant la colonne 'bedrooms' à la place de la colonne 'accommodates' pour faire nos prédictions.
+ Appliquer la fonction au dataframe test_df et assigner l'objet Series résultant contenant les valeurs des prix prédits à la colonne 'predicted_price' de test_df.
+ Calculer l'erreur au carré entre les colonnes 'price' et 'predicted_price' de test_df et assigner l'objet Series résultant à la colonne 'squared_error' de test_df.
+ Calculer la moyenne de la colonne 'squared_error' de test_df et assigner à la variable mse.
+ Afficher la valeur de MSE.


In [7]:
train_df = paris_listings.iloc[0:6000]
test_df = paris_listings.iloc[6000:]

def predict_price(new_listing):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['bedrooms'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbors_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbors_prices.mean()
    return(predicted_price)

test_df['predicted_price'] = test_df['bedrooms'].apply(lambda x: predict_price(x))
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**2
mse = test_df['squared_error'].mean()
print(mse)

6281.181980000002


## Racine carrée de l'erreur quadratique moyenne

+ Calculer la valeur RMSE du modèle entrainé en utilisant la colonne 'bedrooms'.
+ Assigner le résultat à la variable rmse.
+ Afficher le résultat

In [8]:
def predict_price(new_listing):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['bedrooms'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbors_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbors_prices.mean()
    return(predicted_price)

test_df['predicted_price'] = test_df['bedrooms'].apply(lambda x: predict_price(x))
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
mse = test_df['squared_error'].mean()

rmse = mse ** (1/2)
print(rmse)

79.25390829479643


## Comparer l'erreur moyenne absolue et la racine carrée de l'erreur quadratique moyenne

+ Calculer MAE pour errors_one et l'assigner à la variable mae_one.
+ Calculer RMSE pour errors_one et l'assigner à la variable rmse_one.
+ Calculer MAE pour errors_two et l'assigner à mae_two.
+ Calculer RMSE errors_two et l'assigner à rmse_two.


In [9]:
errors_one = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10])
errors_two = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 1000])

mae_one = errors_one.sum()/len(errors_one)
rmse_one = np.sqrt((errors_one**2).sum()/len(errors_one))

mae_two = errors_two.sum()/len(errors_two)
rmse_two = np.sqrt((errors_two**2).sum()/len(errors_two))

In [10]:
print(mae_one)
print(rmse_one)

7.5
7.905694150420948


In [11]:
print(mae_two)
print(rmse_two)

62.5
235.82302686548658
